In [2]:
from typing import Any, Dict, Iterable, List, Optional
import math

def _to_valid_temp(x: Any) -> Optional[float]:
    if x is None:
        return None

    # Convert numeric strings; keep ints/floats as-is
    try:
        val = float(x)
    except (TypeError, ValueError):
        return None

    # Filter out NaN/Inf
    if not math.isfinite(val):
        return None

    # Corruption threshold
    if val < -100:
        return None

    return val


def _extract_candidate_readings(raw: Any) -> Iterable[Any]:
    # Case 1: a plain list/tuple of readings
    if isinstance(raw, (list, tuple)):
        return raw

    # Case 2/3: a dict with either "temps" or "records"
    if isinstance(raw, dict):
        if "temps" in raw:
            return raw.get("temps", [])
        if "records" in raw:
            candidates = []
            for rec in raw.get("records", []):
                # Expect something like [timestamp, temp]
                if isinstance(rec, (list, tuple)) and len(rec) >= 2:
                    candidates.append(rec[1])   # pick the temperature slot
                # else: missing temperature -> ignore
            return candidates

    # Unknown format -> no readings
    return []


def compute_average_temperature(city_data: Dict[str, Any]) -> Dict[str, float]:
    result: Dict[str, float] = {}

    for city, raw in city_data.items():
        # Step A: normalize to candidate values
        candidates = _extract_candidate_readings(raw)

        # Step B: validate/convert each candidate into a float temperature
        valid = []
        for c in candidates:
            v = _to_valid_temp(c)
            if v is not None:
                valid.append(v)

        # Step C: if any valid values, compute rounded mean
        if valid:
            avg = round(sum(valid) / len(valid), 1)
            result[city] = avg
        # else: city omitted per the spec

    return result
city_data = {
    "London": [18.2, "19.7", None, -999, 20.1],
    "Leeds": {"temps": [16.0, 15.4, "error"]},
    "Bristol": {"records": [["2023-07-24T09:00", 18.5],
                            ["2023-07-24T10:00", None],
                            ["2023-07-24T11:00"]]},
    "Oxford": {"records": [["09:00", 21.5], ["10:00", 20.8], ["11:00", "N/A"]]},
    "Hull": [],
    "Bath": {"temps": []}
}

print(compute_average_temperature(city_data))

{'London': 19.3, 'Leeds': 15.7, 'Bristol': 18.5, 'Oxford': 21.1}


In [13]:
import PyPDF2

# Open the PDF file
with open("Puzzle_v1_3.pdf", "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)

    # Extract text from all pages
    text = ""
    for page in reader.pages:
        text += page.extract_text()
import re

# Regex for matching an email address
email_pattern = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-_]+\.[a-zA-Z0-9-.]+"

# Find all matches in the text
emails = re.findall(email_pattern, text)

# Remove duplicates (if needed)
emails = list(set(emails))

print(emails)


['pbinevnapr@kyqf_uip.com', 'regression@star_ngn.com']


In [ ]:
import pandas as pd
# Step 1: Load the dataset
df = pd.read_csv("data.csv")
# Step 2: Remove duplicates
# keep='first' → keep the first occurrence, drop the rest
df_cleaned = df.drop_duplicates(keep='first')
# Step 3: Save cleaned data back
df_cleaned.to_csv("cleaned_data.csv", index=False)

In [ ]:
import pandas as pd

# Read yesterday's and today's CSVs
yesterday = pd.read_csv("yesterday.csv")
today = pd.read_csv("today.csv")

# Merge on unique ID to align rows
merged = pd.merge(yesterday, today, on="id", suffixes=('_yest', '_today'))

# Find rows where any column (except id) has changed
changed_rows = merged[
    (merged.filter(like='_yest') != merged.filter(like='_today')).any(axis=1)
]

# Display changes
for _, row in changed_rows.iterrows():
    print(f"ID: {row['id']}")
    for col in [c for c in merged.columns if '_yest' in c]:
        base_col = col.replace('_yest', '')
        old_val = row[f"{base_col}_yest"]
        new_val = row[f"{base_col}_today"]
        if old_val != new_val:
            print(f" - {base_col}: '{old_val}' → '{new_val}'")
    print()

In [15]:
try:
    result = 10 / 0
except ZeroDivisionError:
    print("Error: Cannot divide by zero.")


Error: Cannot divide by zero.


In [ ]:
"""
step-1: 35/5 = 7 so, at first i need 7 test cases to test all the 35 computer
Step 2: Test the winners of each group (7 computers) to find the fastest overall and rank
them — 2nd fastest overall must be from this race.
Step 3: Test the possible contenders for 2nd & 3rd fastest (the 2nd & 3rd from the
fastest’s group, the 2nd from 2nd place’s group, and the 2nd & 3rd from the final race) —
find top 2 here for final 2nd & 3rd places.
Total tests = 7 (groups) + 2 (winners) + 1 (final) = 10.

"""